In [6]:
import keyring
import os

service_name = 'Open AI API'

#keyring.set_password(service_name, 'personal', # add you key)

api_key = keyring.get_password(service_name, 'personal')

os.environ['OPENAI_API_KEY']=api_key

In [5]:
from langchain.document_loaders import PyPDFDirectoryLoader

download_folder = "data/pdf"

loader = PyPDFDirectoryLoader(download_folder)
docs = loader.load()

Could not import azure.core python package.
C:\Users\Akos Rapp\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Akos Rapp\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\Akos Rapp\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer(3)
incorrect startxref pointer

In [13]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA


embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents([d for d in docs], embeddings)
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=docsearch.as_retriever())


Using embedded DuckDB without persistence: data will be transient


In [24]:
query = "Who tested the skywalk chili 5 xs?"
qa.run(query)

' Philippe Dupont and Claude Thurnheer.'

In [27]:
query = "Give me 2 gliders with a B in big ears"
qa.run(query)

' The glider model Base2 Lite S with serial number BG0837386A has a B classification in big ears, as does the glider model Base2 Lite ML with serial number BG0910117A.'

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings # does not seem to be good?

# https://www.sbert.net/docs/pretrained_models.html
# https://huggingface.co/spaces/mteb/leaderboard

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
hf = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs) 

#hf.embed_query("what is in the documents")

In [29]:
from langchain.vectorstores import FAISS

# this can take some time
db = FAISS.from_documents([d for d in docs], hf)
db.save_local("data/faiss_index")

In [32]:
db2 = FAISS.load_local("data/faiss_index", hf)

query = "skywalk chili m"
docs = db2.similarity_search(query,k=5)

docs

[Document(page_content='Flight test report: EN 926-2:2013 & NfL 2-565-20\nManufacturer Skywalk GmbH & Co. KG Certification number PG_1817.2021\nAddress Windeckstr. 4\n83250 Marquartstein\nGermanyFlight test 16.06.2021\nGlider model Chili 5 XXS Classification B\nSerial number CH10 004 Representative None\nTrimmer no Place of test Villeneuve\nFolding lines used no \nTest pilot Light pilot under Air\nTurquoise supervisionClaude Thurnheer\nHarness Flugsau - XX-Lite Advance - Success 4 M\nHarness to risers distance (cm) 40 44\nDistance between risers (cm) 40 44\nTotal weight in flight (kg) 55 85\nTest Report generated automatically by AIR TURQUOISE SA, valid without signature Rev 06 | 19.02.2021   //   ISO | 91.22  //  Page 1 of 31. Inflation/Take-off A \nRising behaviour Smooth, easy and constant rising ASmooth, easy and constant rising A\nSpecial take off technique required No ANo A\n2. Landing A \nSpecial landing technique required No ANo A\n3. Speed in straight flight B \nTrim speed mor